In [71]:
from ams_paq_utilities import *
from ams_utilities import *
import scipy
import matplotlib.pyplot as plt
%matplotlib notebook
import pyabf
import paq2py
import numpy as np
import pandas as pd
import statistics

In [72]:
# Import data from spreadsheet 
# df_all = GS_to_df('https://docs.google.com/spreadsheets/d/1ziOx80em0ZhmMmSjKePYbOq3K6sHcOapfHjy9S4oDbk/edit#gid=662691853')
df_all=pd.read_csv(r'E:\Jake\Morph_analysis.csv')
df_all.head()

,Mouse#: Cell,AS016: Cell1,AS016: Cell2,AS029: Cell1,AS030: Cell1,AS035: Cell1,AS036: Cell1,AS038: Cell1,AS039: Cell1,AS039: Cell2,...,AS276: Cell2,AS277: Cell1,AS277: Cell2,AS277: Cell3,AS278: Cell2,AS278: Cell3,AS278: Cell4,AS278: Cell5,AS279: Cell1,AS280: Cell2
0,Slice_Orientation,H,H,H,H,H,C,H,H,H,...,C,C,C,C,C,C,C,C,C,C
1,Axon_node_total,6,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Axon_length,1620.49,0,28.4,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Axon_SA,10859.8,0,104.62,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Axon_length_to_SA_ratio,0.1492191385,NaN,0.2714586121,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [73]:
# Extents and oreintations df
extents_all=df_all.iloc[61,:]
orientations_all=df_all.iloc[0,:]
df=pd.concat([extents_all,orientations_all],1)
df=df.drop('Mouse#: Cell',axis=0)
df

,61,0
AS016: Cell1,400,H
AS016: Cell2,130,H
AS029: Cell1,190,H
AS030: Cell1,420,H
AS035: Cell1,110,H
AS036: Cell1,460,C
AS038: Cell1,230,H
AS039: Cell1,300,H
AS039: Cell2,70,H
AS040: Cell1,200,H


In [74]:
# Make lists of horizontal and coronal extents
h_extents=[]
c_extents=[]
extents_str=list(df.iloc[:,0])
extents=[]
for i in extents_str:
    i=int(i)
    extents.append(i)

for i in range(0,len(extents)):
    if df.iat[i,1]=='H':
        h_extents.append(extents[i])
    elif df.iat[i,1]=='C':
        c_extents.append(extents[i])
    else:
        print('Error')

In [75]:
#Are the data normally distributed? p>0.05 suggests yes. Kolmogorov-Smirnov test
h_mean=statistics.mean(h_extents)
h_std=statistics.stdev(h_extents)
print('Horizontal - ' + str(scipy.stats.kstest(h_extents, 'norm', args=(h_mean,h_std))))

c_mean=statistics.mean(c_extents)
c_std=statistics.stdev(c_extents)
print('Coronal - ' + str(scipy.stats.kstest(c_extents, 'norm', args=(c_mean,c_std))))

Horizontal - KstestResult(statistic=0.1997992793069301, pvalue=0.3555669090045612)
Coronal - KstestResult(statistic=0.1097680893699699, pvalue=0.37925553988814026)


In [76]:
#T-test, since normal distribution can be assumed
scipy.stats.ttest_ind(h_extents, c_extents, equal_var=False)

Ttest_indResult(statistic=-1.2329478614584781, pvalue=0.22579070007038013)

In [77]:
#D'Agostino normality test
print('DAgostino\nHorizontal - '+str(scipy.stats.normaltest(h_extents)))
print('Coronal - '+str(scipy.stats.normaltest(c_extents))+'\n')

#Shapiro-Wilk normality test
print('Shapiro-Wilk\nHorizontal - '+str(scipy.stats.shapiro(h_extents)))
print('Coronal - '+str(scipy.stats.shapiro(c_extents)))

DAgostino
Horizontal - NormaltestResult(statistic=1.8687170091241527, pvalue=0.39283778836828626)
Coronal - NormaltestResult(statistic=3.790620944625695, pvalue=0.15027167260932722)

Shapiro-Wilk
Horizontal - (0.9346175789833069, 0.18939079344272614)
Coronal - (0.9594244956970215, 0.030021414160728455)


In [78]:
#Mann-Whitney U test
scipy.stats.mannwhitneyu(h_extents,c_extents)

MannwhitneyuResult(statistic=563.5, pvalue=0.1630093494527094)

In [116]:
df_thickness=pd.DataFrame(df_all.iloc[28,1:])
df_thickness=df_thickness.drop(['AS035: Cell1',
                    'AS039: Cell2',
                    'AS057: Cell3',
                    'AS062: Cell3',
                    'AS063: Cell3',
                    'AS091: Cell3',
                    'AS132: Cell3',
                    'AS143: Cell3'],0)
df_confocal=df_thickness.iloc[:53].values
df_confocal_fl=[]
for i in df_thickness.iloc[:53].values:
    for j in i:
        df_confocal_fl.append(float(j)) 

df_2P_=df_thickness.iloc[54:].values
df_2P_fl=[]
for i in df_thickness.iloc[54:].values:
    for j in i:
        df_2P_fl.append(float(j))

df_confocal=df_confocal_fl
df_2P=df_2P_fl

In [117]:
df_2P_mean=statistics.mean(df_2P)
df_2P_std=statistics.stdev(df_2P)
scipy.stats.ttest_ind(df_confocal,df_2P, equal_var=False)


Ttest_indResult(statistic=-5.246484733318764, pvalue=1.1815840355195495e-05)

In [118]:
scipy.stats.kstest(df_2P, 'norm', args=(statistics.mean(df_2P),statistics.stdev(df_2P)))

KstestResult(statistic=0.13072861498141203, pvalue=0.806661688590387)

In [119]:
print(statistics.mean(df_confocal))
print(statistics.mean(df_2P))

11921.821509433963
26669.325


[28670.5, 12011.6, 16403.2, 54124.2, 28332.4, 19268.6, 61809.9, 35952.9, 14822.4, 22051.0, 14403.0, 14937.6, 12372.2, 25540.2, 32822.1, 32575.6, 20255.8, 22371.5, 12824.3, 33066.0, 30862.2, 39934.7, 18904.8, 35747.1]


array([['28670.5'],
       ['12011.6'],
       ['16403.2'],
       ...,
       ['39934.7'],
       ['18904.8'],
       ['35747.1']], dtype=object)